In [1]:
!pip install --upgrade tensorflow --user
!pip install tf-quant-finance
!pip install QuantLib-Python

ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
You should consider upgrading via the 'C:\Dev\github\tf-quant-finance\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Dev\github\tf-quant-finance\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Dev\github\tf-quant-finance\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
# reduce number of threads
os.environ['TF_NUM_INTEROP_THREADS'] = '1'
os.environ['TF_NUM_INTRAOP_THREADS'] = '1'

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tf_quant_finance as tff 
import tensorflow as tf
import functools
import pandas as pd
import time
import QuantLib as ql

In [3]:
!nvidia-smi

Tue May  3 09:18:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   73C    P0    N/A /  N/A |      0MiB /  2048MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [62]:
spot = 20.0
strike = 20.0
K_lower = 15.0
K_upper = 20.0
K_knockout = 30.0
tarf_target = 5.0
step_up_ratio = 2.0

r = 0.0
volatility = 0.5

In [63]:
#@title Set up parameters

dtype = tf.float64 #@param
num_samples = 200000#200000 #@param
num_timesteps = 53 #@param

# expiries =tf.constant( [0.0, 0.5, 1.0], dtype=dtype) # This can be a rank 1 Tensor
dt = 1. / num_timesteps
# times = [1.0]
times = tf.linspace(tf.constant(0.0, dtype=dtype), tf.constant(1.0, dtype=dtype), num_timesteps)
rate = tf.constant(r, dtype=dtype)
sigma = tf.constant(volatility, dtype=dtype)
spot = tf.constant(spot, dtype=dtype)
strikes = tf.constant(strike, dtype=dtype)

def set_up_pricer(times, watch_params=False):
    """Set up European option pricing function under Black-Scholes model.
    
    Args:
        expiries: List of expiries at which to to sample the trajectories.
        watch_params: A Python bool. When `True`, gradients of the price function wrt the inputs
          are computed more efficiently. 
    Returns:
     A callable that accepts a rank 1 tensor of strikes, and scalar values for 
     the spots and  volatility values. The callable outputs prices of
     the European call options on the grid `expiries x strikes`.
    """
    def price_eu_options(strikes, spot, sigma):
        # Define drift and volatility functions. 
        def drift_fn(t, x):
          del t, x
          return rate - 0.5 * sigma**2
        def vol_fn(t, x):
          del t, x
          return tf.reshape(sigma, [1, 1])
        # Use GenericItoProcess class to set up the Ito process
        process = tff.models.GenericItoProcess(
            dim=1,
            drift_fn=drift_fn,
            volatility_fn=vol_fn,
            dtype=dtype)
        log_spot = tf.math.log(tf.reduce_mean(spot))
        if watch_params:
            watch_params_list = [sigma]
        else:
            watch_params_list = None
        paths = process.sample_paths(
            times=times, num_samples=num_samples,
            initial_state=log_spot, 
            watch_params=watch_params_list,
            # Select a random number generator
            random_type=tff.math.random.RandomType.SOBOL, #PSEUDO_ANTITHETIC
            time_step=dt)
        
#         tf.print('paths', paths.shape, paths)
        # paths_reduced = tf.gather_nd(paths, [[:, 0]])
        paths_reduced = tf.expand_dims(paths[:, -1, :], axis=-1)
        # tf.print('paths_reduced', paths_reduced.shape, paths_reduced)
        # tf.print('strikes', strikes.shape, strikes)
        # tf.print('tf.math.exp(paths) - strikes', (tf.math.exp(paths) - strikes).shape, tf.math.exp(paths) - strikes)

#         tf.print('paths', paths.shape, paths)
#         tf.print('paths', tf.reshape(paths, [num_samples, num_timesteps]).shape, tf.reshape(paths, [num_samples, num_timesteps]))
        
        @tf.function
        def my_function(element):
#             tf.print(element, summarize=-1)
            total = tf.constant(0.0, dtype=tf.float64)
            discounted_payoff = tf.constant(0.0, dtype=tf.float64)
            df = tf.constant(1.0, dtype=tf.float64)
            is_active = True
            cashflow = tf.constant(0.0, dtype=tf.float64)
            for cur_spot in element:
                if is_active:
                    if K_knockout <= cur_spot:
                        # early termination
                        cashflow = tf.constant(0.0, dtype=tf.float64)
                        is_active = False
                    if K_upper <= cur_spot: # cur_spot < K_knockout
                        cashflow = cur_spot - strike
                    if cur_spot < K_lower:
                        cashflow = step_up_ratio*(cur_spot - strike)

                    if total + cashflow >= tarf_target:
                        cashflow = tarf_target - total
                        total += cashflow
                        discounted_payoff += df*cashflow
                        is_active = False
                    else:
                        total += cashflow
                        discounted_payoff += df*cashflow
                     
            return discounted_payoff

        payoffs = tf.map_fn(my_function, tf.reshape(tf.math.exp(paths), [num_samples, num_timesteps]))
#         tf.print(payoffs, summarize=-1)
        
        prices = tf.reduce_mean(payoffs)

#         prices = (tf.exp(-tf.expand_dims(rate * times[-1], axis=-1))
#                   * tf.reduce_mean(tf.nn.relu(tf.math.exp(paths_reduced) - strikes), 0))
        # tf.print(prices.shape, prices)
        return prices
    return price_eu_options

price_eu_options = tf.function(set_up_pricer(times, watch_params=True),
                               input_signature=[
                                                tf.TensorSpec([None], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64)
                               ])
price_eu_options_xla = tf.function(set_up_pricer(times, watch_params=True),
                               input_signature=[
                                                tf.TensorSpec([None], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64)
                               ], jit_compile=True)
# price_eu_options_xla = tf.function(set_up_pricer(times), jit_compile=True)
print(set_up_pricer(times)(strikes, spot, sigma))

# price_eu_options_xla = set_up_pricer(times)

KeyboardInterrupt: 